# 1. Cargar datos preparados y separar target

In [43]:
import pandas as pd

df = pd.read_csv('Datasets/datos_preparados.csv')

X = df.drop('target',axis=1)
y = df['target']

print(X.shape)
print(y.shape)

(4401, 69)
(4401,)


# 2. Selección manual

In [45]:
#Dataframe sin dummies
df2 = pd.read_csv('Datasets/datos_limpios.csv').drop(['attrition','employee_id'], axis=1)


De las variables "Años trabajando", "Años bajo el mando del mismo jefe", "Años desde la última promoción" y "Años en la empresa", incluir solo una de estas, ya que presentan una alta correlación entre ellas. Se deja solo "Años en la empresa".

In [46]:
df_manual = df2.drop(['years_since_last_promotion', 'total_working_years', 'years_with_curr_manager'], axis=1)

Excluir también las variables categóricas que no presentan correlación con la variable objetivo.

In [47]:
df_manual = df_manual.drop(['stock_option_level', 'gender', 'education', 'job_level', 'performance_rating'], axis=1)

Excluir variable numéricas que no presentan correlación con la variable objetivo.

In [48]:
df_manual = df_manual.drop(['distance_from_home', 'percent_salary_hike', 'training_times_last_year'], axis=1)

In [49]:
df_manual = pd.get_dummies(df_manual)

# 3. Variance Threshold

In [24]:
from sklearn.feature_selection import VarianceThreshold

#Función de filtro de caracteristicas
def variance_threshold(X,th):
    var_thres=VarianceThreshold(threshold=th)
    var_thres.fit(X)
    new_cols = var_thres.get_support()
    return new_cols

In [25]:
# Obtener columnas seleccionadas
X_new = variance_threshold(X, 0.20)
# Nuevo dataframe
df_vth = X.iloc[:,X_new]
df_vth.head()

,age,distance_from_home,monthly_income,num_companies_worked,percent_salary_hike,total_working_years,training_times_last_year,years_at_company,years_since_last_promotion,years_with_curr_manager,...,marital_status_Married,marital_status_Single,stock_option_level_0,stock_option_level_1,environment_satisfaction_Alto,environment_satisfaction_Muy alto,job_satisfaction_Alto,job_satisfaction_Muy alto,work_life_balance_Muy buena,job_involvement_Alto
0,51,6,131160.0,1.0,11,1.0,6,1.0,0.0,0,...,True,False,True,False,True,False,False,True,False,True
1,31,10,41890.0,0.0,23,6.0,3,5.0,1.0,4,...,False,True,False,True,True,False,False,False,False,False
2,32,17,49185.0,1.0,15,5.0,2,5.0,0.0,3,...,True,False,False,False,False,False,False,False,False,True
3,38,2,83210.0,3.0,11,13.0,5,8.0,7.0,5,...,True,False,False,False,False,True,False,True,True,False
4,32,10,23420.0,4.0,12,9.0,2,6.0,0.0,4,...,False,True,False,False,False,True,False,False,True,True


In [26]:
# Obtener columnas seleccionadas
X_new = variance_threshold(X, 0.25)
# Nuevo dataframe
df_vth2 = X.iloc[:,X_new]
df_vth2.head()

,age,distance_from_home,monthly_income,num_companies_worked,percent_salary_hike,total_working_years,training_times_last_year,years_at_company,years_since_last_promotion,years_with_curr_manager,mean_time
0,51,6,131160.0,1.0,11,1.0,6,1.0,0.0,0,6.8702
1,31,10,41890.0,0.0,23,6.0,3,5.0,1.0,4,7.3160
2,32,17,49185.0,1.0,15,5.0,2,5.0,0.0,3,6.8161
3,38,2,83210.0,3.0,11,13.0,5,8.0,7.0,5,6.7892
4,32,10,23420.0,4.0,12,9.0,2,6.0,0.0,4,7.8776


# 4. SelectKBest

In [27]:
# Para clasificación
from sklearn.feature_selection import SelectKBest, f_classif,chi2

# Función de filtro de caracteristicas - stadis. scores
def select_kbest_classification(X,y,score_f,k):
    sel_kb = SelectKBest(score_func=score_f, k=k)
    sel_kb.fit(X,y)
    new_cols = sel_kb.get_support()
    print("Scores:\n", sel_kb.scores_, "\nP-values:\n", sel_kb.pvalues_)
    return new_cols

In [28]:
# Obtener columnas seleciconadas - (30 caracteristicas)
X_new = select_kbest_classification(X, y, f_classif, 30)
# Nuevo conjunto de datos
df_anova = X.iloc[:,X_new]
df_anova.head()

Scores:
 [1.14674021e+02 3.26032102e-01 1.85064922e+00 7.97752451e+00
 4.82372552e+00 1.56475671e+02 1.03410179e+01 1.41173445e+02
 3.27914499e+00 1.09151201e+02 1.79738311e+02 2.43502944e+01
 5.84619038e+01 1.06625482e+01 2.73403550e+01 9.79276958e-01
 1.61003664e+00 2.56629853e-01 8.01023620e-01 2.93364724e-01
 2.83997308e-01 5.67347842e+00 3.73402097e+01 7.06112748e-01
 5.91701394e-02 4.20378146e-04 2.95553067e+00 7.26231807e+00
 1.51276719e+00 1.51276719e+00 8.15091699e-01 5.06978567e+00
 1.12550347e+00 1.32294557e-03 1.51121020e+00 8.23515065e-01
 8.39184899e-01 1.10559621e-03 1.38338151e+00 8.87092588e+00
 1.00409332e+01 3.37123487e+00 5.62906949e-01 4.93832845e-01
 3.35925584e+01 3.68383474e+01 1.38798269e+02 1.28457502e+00
 2.62603388e+00 1.06608136e+00 1.33260451e-01 7.55391461e+00
 6.52922036e+01 1.16763561e+00 1.01058983e+01 2.57592067e-01
 3.68278854e+01 4.35373689e-02 3.43361407e+01 4.41849999e-01
 1.12269180e+00 4.40242180e+01 1.86114692e+01 3.27461546e+00
 6.41366201e+00

,age,num_companies_worked,percent_salary_hike,total_working_years,training_times_last_year,years_at_company,years_with_curr_manager,mean_time,business_travel_Non_Travel,business_travel_Travel_Frequently,...,marital_status_Married,marital_status_Single,environment_satisfaction_Alto,environment_satisfaction_Bajo,environment_satisfaction_Muy alto,job_satisfaction_Bajo,job_satisfaction_Muy alto,work_life_balance_Mala,work_life_balance_Muy buena,job_involvement_Bajo
0,51,1.0,11,1.0,6,1.0,0,6.8702,False,False,...,True,False,True,False,False,False,True,False,False,False
1,31,0.0,23,6.0,3,5.0,4,7.3160,False,True,...,False,True,True,False,False,False,False,False,False,False
2,32,1.0,15,5.0,2,5.0,3,6.8161,False,True,...,True,False,False,False,False,False,False,True,False,False
3,38,3.0,11,13.0,5,8.0,5,6.7892,True,False,...,True,False,False,False,True,False,True,False,True,False
4,32,4.0,12,9.0,2,6.0,4,7.8776,False,False,...,False,True,False,False,True,True,False,False,True,False


In [29]:
# Obtener columnas seleciconadas - (30 caracteristicas)
X_new = select_kbest_classification(X, y, chi2, 30)
# Nuevo conjunto de datos
df_chi2 = X.iloc[:,X_new]
df_chi2.head()

Scores:
 [2.52902718e+02 2.32942732e+00 5.51997097e+04 1.83980234e+01
 4.24352197e+00 7.16106872e+02 6.12232584e+00 4.82032482e+02
 9.78900381e+00 3.28957950e+02 4.18613543e+01 2.17555376e+01
 4.68486158e+01 3.09019941e+00 2.60226070e+01 3.39187526e-01
 1.12138141e+00 2.48331338e-01 4.89738860e-01 2.14057278e-01
 2.51184858e-01 4.58033708e+00 3.63609864e+01 4.15510913e-01
 5.27802919e-02 2.87833338e-04 2.78972149e+00 6.60096512e+00
 9.07554551e-01 6.05380138e-01 5.14455631e-01 3.22554822e+00
 9.58696972e-01 1.22791206e-03 1.44029075e+00 7.50177449e-01
 8.09652296e-01 9.11570003e-04 1.28737585e+00 7.98768962e+00
 9.47833429e+00 2.69936411e+00 4.38087473e-01 4.66164492e-01
 2.59413257e+01 1.98233408e+01 9.15167215e+01 7.33328131e-01
 1.56131634e+00 9.51705615e-01 1.25592429e-01 5.19244920e+00
 5.20226629e+01 9.41503028e-01 7.03430639e+00 1.80520537e-01
 2.93986930e+01 3.52630393e-02 2.33588875e+01 3.39665064e-01
 1.00707749e+00 4.12396571e+01 7.21268392e+00 1.34041094e+00
 6.04763911e+00

,age,monthly_income,num_companies_worked,percent_salary_hike,total_working_years,training_times_last_year,years_at_company,years_since_last_promotion,years_with_curr_manager,mean_time,...,marital_status_Married,marital_status_Single,environment_satisfaction_Alto,environment_satisfaction_Bajo,environment_satisfaction_Muy alto,job_satisfaction_Bajo,job_satisfaction_Muy alto,work_life_balance_Mala,work_life_balance_Muy buena,job_involvement_Bajo
0,51,131160.0,1.0,11,1.0,6,1.0,0.0,0,6.8702,...,True,False,True,False,False,False,True,False,False,False
1,31,41890.0,0.0,23,6.0,3,5.0,1.0,4,7.3160,...,False,True,True,False,False,False,False,False,False,False
2,32,49185.0,1.0,15,5.0,2,5.0,0.0,3,6.8161,...,True,False,False,False,False,False,False,True,False,False
3,38,83210.0,3.0,11,13.0,5,8.0,7.0,5,6.7892,...,True,False,False,False,True,False,True,False,True,False
4,32,23420.0,4.0,12,9.0,2,6.0,0.0,4,7.8776,...,False,True,False,False,True,True,False,False,True,False


# 5. RFE

In [30]:
from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier

# Función recursiva de selección de características
def recursive_feature_selection(X,y,model,k):
  rfe = RFE(model, n_features_to_select=k, step=1)
  fit = rfe.fit(X, y)
  X_new = fit.support_
  print("Num Features: %s" % (fit.n_features_))
  print("Selected Features: %s" % (fit.support_))
  print("Feature Ranking: %s" % (fit.ranking_))

  return X_new

In [31]:
# Establecer Estimador
model = LogisticRegression(max_iter=1000,random_state=24)
# Obtener columnas seleccionadas - (30 caracteristicas)
X_new = recursive_feature_selection(X, y, model, 30)
# Nuevo conjunto de datos
df_log = X.iloc[:,X_new]
df_log.head()

Num Features: 30
Selected Features: [False False False False False False False False False False  True  True
  True False  True  True  True False False False False  True  True False
 False False  True  True False False False  True False False  True False
  True  True False  True  True  True  True False  True  True  True False
  True False False False  True False  True False  True False  True False
 False  True  True  True  True False False False False]
Feature Ranking: [30 39 40 11 37 12  3 18 16 25  1  1  1 23  1  1  1  9 31 32 27  1  1 14
 19 38  1  1 22 13 17  1 34 28  1 26  1  1  5  1  1  1  1 10  1  1  1 21
  1 15  2  6  1  7  1 24  1 36  1  4  8  1  1  1  1 33 29 20 35]


,mean_time,business_travel_Non_Travel,business_travel_Travel_Frequently,department_Human Resources,department_Research & Development,department_Sales,education_Universitario,education_field_Human Resources,education_field_Other,education_field_Technical Degree,...,marital_status_Single,stock_option_level_1,environment_satisfaction_Bajo,environment_satisfaction_Muy alto,job_satisfaction_Bajo,job_satisfaction_Muy alto,work_life_balance_Mala,work_life_balance_Muy buena,job_involvement_Alto,job_involvement_Bajo
0,6.8702,False,False,False,False,True,True,False,False,False,...,False,False,False,False,False,True,False,False,True,False
1,7.3160,False,True,False,True,False,False,False,False,False,...,True,True,False,False,False,False,False,False,False,False
2,6.8161,False,True,False,True,False,False,False,True,False,...,False,False,False,False,False,False,True,False,True,False
3,6.7892,True,False,False,True,False,False,False,False,False,...,False,False,False,True,False,True,False,True,False,False
4,7.8776,False,False,False,True,False,False,False,False,False,...,True,False,False,True,True,False,False,True,True,False


In [32]:
# Establecer Estimador
model = RandomForestClassifier(n_estimators=300,random_state=24)
# Obtener columnas seleciconadas - (30 caracteristicas)
X_new = recursive_feature_selection(X, y, model, 30)
# Nuevo conjunto de datos
df_ranfor = X.iloc[:,X_new]
df_ranfor.head()

Num Features: 30
Selected Features: [ True  True  True  True  True  True  True  True  True  True  True False
  True False False  True False False  True False False False False  True
 False False False False False  True  True  True False False False False
 False False False False False False  True False False  True  True  True
  True False False False  True False  True  True  True False  True False
 False False  True  True False False False False False]
Feature Ranking: [ 1  1  1  1  1  1  1  1  1  1  1 31  1  7 14  1  9 40  1 10 23  8 27  1
 28  6 37 36  3  1  1  1 18 30 39 32 38 16 35 34 20 11  1 29 13  1  1  1
  1 22 33  4  1 19  1  1  1 15  1 12 24  2  1  1 25  5 17 21 26]


,age,distance_from_home,monthly_income,num_companies_worked,percent_salary_hike,total_working_years,training_times_last_year,years_at_company,years_since_last_promotion,years_with_curr_manager,...,marital_status_Single,stock_option_level_0,stock_option_level_1,environment_satisfaction_Bajo,environment_satisfaction_Muy alto,job_satisfaction_Alto,job_satisfaction_Bajo,job_satisfaction_Muy alto,work_life_balance_Muy buena,job_involvement_Alto
0,51,6,131160.0,1.0,11,1.0,6,1.0,0.0,0,...,False,True,False,False,False,False,False,True,False,True
1,31,10,41890.0,0.0,23,6.0,3,5.0,1.0,4,...,True,False,True,False,False,False,False,False,False,False
2,32,17,49185.0,1.0,15,5.0,2,5.0,0.0,3,...,False,False,False,False,False,False,False,False,False,True
3,38,2,83210.0,3.0,11,13.0,5,8.0,7.0,5,...,False,False,False,False,True,False,False,True,True,False
4,32,10,23420.0,4.0,12,9.0,2,6.0,0.0,4,...,True,False,False,False,True,False,True,False,True,True


# 6. Select From model

In [33]:
from sklearn.feature_selection import SelectFromModel
from sklearn.linear_model import RidgeClassifier

# Selector de variables con Ridge
sel_ = SelectFromModel(RidgeClassifier(alpha=0.1,random_state=24), max_features=30)
sel_.fit(X, y)
print(sel_.estimator_.coef_)
#Obtener variables seleccionadas
X_new = sel_.get_support()

df_ridge = X.iloc[:,X_new]
df_ridge.head()

[[-6.99331594e-03 -7.84264695e-04 -1.03785765e-07  2.59951636e-02
   4.56601675e-03 -1.09785962e-02 -2.96420802e-02 -1.18212188e-02
   2.27607839e-02 -7.20751955e-03  1.09970811e-01 -1.45266184e-01
   1.71027204e-01 -2.57610201e-02  1.11603166e-01 -5.52713024e-02
  -5.63318634e-02 -6.45421509e-02 -9.07841798e-03 -3.97773776e-03
   2.21690046e-02  5.54293020e-02  2.03124366e-01  9.02033084e-03
  -3.85744601e-02 -1.91964760e-02 -8.59702707e-02 -6.84034899e-02
  -6.60052997e-03  6.60052997e-03  2.41715345e-02  5.49087703e-02
  -1.79648653e-03 -1.01183705e-02 -6.71654478e-02 -4.03054996e-02
  -1.12140632e-02  2.34732004e-02 -3.89799712e-02 -1.11029572e-01
   1.57043365e-01  3.31997851e-02  4.89752538e-02 -6.11624986e-02
  -9.12792223e-02 -6.62322287e-02  1.57511451e-01  1.72094217e-02
  -2.92382805e-02  2.49448754e-02 -1.29160166e-02 -4.12839705e-02
   1.71858931e-01 -3.43888039e-02 -9.61861569e-02  1.37620684e-03
   1.32520869e-01 -1.46520253e-02 -1.19245050e-01 -5.38122191e-02
  -3.65486

,mean_time,business_travel_Non_Travel,business_travel_Travel_Frequently,department_Human Resources,department_Research & Development,department_Sales,education_Doctor,education_Universitario,education_field_Human Resources,education_field_Other,...,marital_status_Married,marital_status_Single,environment_satisfaction_Bajo,environment_satisfaction_Muy alto,job_satisfaction_Bajo,job_satisfaction_Muy alto,work_life_balance_Buena,work_life_balance_Mala,work_life_balance_Muy buena,job_involvement_Bajo
0,6.8702,False,False,False,False,True,False,True,False,False,...,True,False,False,False,False,True,True,False,False,False
1,7.3160,False,True,False,True,False,False,False,False,False,...,False,True,False,False,False,False,False,False,False,False
2,6.8161,False,True,False,True,False,False,False,False,True,...,True,False,False,False,False,False,False,True,False,False
3,6.7892,True,False,False,True,False,True,False,False,False,...,True,False,False,True,False,True,False,False,True,False
4,7.8776,False,False,False,True,False,False,False,False,False,...,False,True,False,True,True,False,False,False,True,False


In [34]:
from sklearn.ensemble import GradientBoostingClassifier

# Selector de variables con Lasso
sel_ = SelectFromModel(GradientBoostingClassifier(n_estimators=300,random_state=24))
sel_.fit(X, y)
print(sel_.estimator_.feature_importances_)
#Obtener variables seleccionadas
X_new = sel_.get_support()

df_gb = X.iloc[:,X_new]
df_gb.head()

[0.10401526 0.03787741 0.08244816 0.04088604 0.0267724  0.08512643
 0.02665626 0.05513782 0.04018762 0.06221201 0.13383995 0.00364211
 0.02502195 0.         0.00832794 0.00016459 0.00317739 0.00081577
 0.         0.0009833  0.0026236  0.00429578 0.0076358  0.00632725
 0.00048739 0.00238025 0.00047639 0.00216547 0.00059716 0.00214205
 0.00311649 0.00505224 0.00266181 0.00313631 0.00246332 0.00023794
 0.00710266 0.00062837 0.00314921 0.00559889 0.00782621 0.00258533
 0.00591841 0.00454272 0.00860333 0.00230796 0.03527031 0.00353724
 0.00272651 0.00457232 0.0027012  0.00290606 0.03132978 0.00177866
 0.00933774 0.00249808 0.02037487 0.0016294  0.01521114 0.0029059
 0.00326307 0.01187225 0.00486189 0.00229819 0.00214221 0.00183657
 0.00503291 0.00019363 0.00036537]


,age,distance_from_home,monthly_income,num_companies_worked,percent_salary_hike,total_working_years,training_times_last_year,years_at_company,years_since_last_promotion,years_with_curr_manager,mean_time,business_travel_Travel_Frequently,marital_status_Single,environment_satisfaction_Bajo,job_satisfaction_Bajo,job_satisfaction_Muy alto
0,51,6,131160.0,1.0,11,1.0,6,1.0,0.0,0,6.8702,False,False,False,False,True
1,31,10,41890.0,0.0,23,6.0,3,5.0,1.0,4,7.3160,True,True,False,False,False
2,32,17,49185.0,1.0,15,5.0,2,5.0,0.0,3,6.8161,True,False,False,False,False
3,38,2,83210.0,3.0,11,13.0,5,8.0,7.0,5,6.7892,False,False,False,False,True
4,32,10,23420.0,4.0,12,9.0,2,6.0,0.0,4,7.8776,False,True,False,True,False


# Modelos 

In [68]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier,GradientBoostingClassifier
from sklearn.svm import SVC
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt

## Regresión logística

In [120]:
def regresionLogistica (df,y):
    X_train, X_test,y_train,y_test = train_test_split(df,y,test_size=0.2, random_state=24)

    num=df.select_dtypes(include=[float,int]).columns
    pipeline=ColumnTransformer([('num',StandardScaler(),num)],remainder='passthrough')
    X_train_std=pipeline.fit_transform(X_train)
    X_test_std=pipeline.transform(X_test)

    lr=LogisticRegression(max_iter=1000,class_weight="balanced").fit(X_train_std,y_train)
    y_pred_train=lr.predict(X_train_std)
    y_pred_test=lr.predict(X_test_std)
    mc_train=confusion_matrix(y_train,y_pred_train)
    mc_test=confusion_matrix(y_test,y_pred_test)
    tn, fp, fn, tp = mc_train.ravel()

    precision = tp / (tp + fp)
    recall = tp / (tp + fn)
    especificidad = tn / (fp + tn)
    f1_score = 2*(precision*recall)/(precision+recall)
    print('-'*30,'TRAIN','-'*30)
    print(f'Precision: {precision}')
    print(f'Recall: {recall}')
    print(f'Especificidad: {especificidad}')
    print(f'F1 score: {f1_score}')
    print('Train score: ',lr.score(X_train_std,y_train))

    tn, fp, fn, tp = mc_test.ravel()

    precision = tp / (tp + fp)
    recall = tp / (tp + fn)
    especificidad = tn / (fp + tn)
    f1_score = 2*(precision*recall)/(precision+recall)
    print('-'*30,'TEST','-'*30)
    print(f'Precision : {precision}')
    print(f'Recall : {recall}')
    print(f'Especificidad : {especificidad}')
    print(f'F1 score : {f1_score}')
    print('Test score: ',lr.score(X_test_std,y_test))

### Regresión logística con todas las características

In [121]:
regresionLogistica(X,y)

------------------------------ TRAIN ------------------------------
Precision: 0.36257309941520466
Recall: 0.7722419928825622
Especificidad: 0.7420554428668019
F1 score: 0.4934621944286526
Train score:  0.746875
------------------------------ TEST ------------------------------
Precision : 0.348993288590604
Recall : 0.7074829931972789
Especificidad : 0.7356948228882834
F1 score : 0.46741573033707867
Test score:  0.7309875141884222


### Regresión logística con selección manual

In [122]:
regresionLogistica(df_manual,y)

------------------------------ TRAIN ------------------------------
Precision: 0.35401157981803144
Recall: 0.7615658362989324
Especificidad: 0.7359702501690332
F1 score: 0.48334274421230944
Train score:  0.7400568181818182
------------------------------ TEST ------------------------------
Precision : 0.36721311475409835
Recall : 0.7619047619047619
Especificidad : 0.7370572207084468
F1 score : 0.4955752212389381
Test score:  0.7412031782065834


### Regresión logística con Variance threshold 0.20

In [123]:
regresionLogistica(df_vth,y)

------------------------------ TRAIN ------------------------------
Precision: 0.33601933924254634
Recall: 0.7419928825622776
Especificidad: 0.7214334009465855
F1 score: 0.46256239600665555
Train score:  0.7247159090909091
------------------------------ TEST ------------------------------
Precision : 0.3193548387096774
Recall : 0.673469387755102
Especificidad : 0.7125340599455041
F1 score : 0.4332603938730853
Test score:  0.7060158910329172


### Regresión logística con Variance threshold 0.25

In [124]:
regresionLogistica(df_vth2,y)

------------------------------ TRAIN ------------------------------
Precision: 0.29596412556053814
Recall: 0.7046263345195729
Especificidad: 0.6815415821501014
F1 score: 0.4168421052631579
Train score:  0.6852272727272727
------------------------------ TEST ------------------------------
Precision : 0.2761627906976744
Recall : 0.6462585034013606
Especificidad : 0.6607629427792916
F1 score : 0.3869653767820774
Test score:  0.6583427922814983


### Regresión logística con SelectKbest Anova

In [125]:
regresionLogistica(df_anova,y)

------------------------------ TRAIN ------------------------------
Precision: 0.3453355155482815
Recall: 0.7508896797153025
Especificidad: 0.7295469912102772
F1 score: 0.4730941704035875
Train score:  0.7329545454545454
------------------------------ TEST ------------------------------
Precision : 0.36038961038961037
Recall : 0.7551020408163265
Especificidad : 0.7316076294277929
F1 score : 0.4879120879120879
Test score:  0.735527809307605


### Regresión logística con SelectKbest chi2

In [126]:
regresionLogistica(df_chi2,y)

------------------------------ TRAIN ------------------------------
Precision: 0.35449299258037925
Recall: 0.7651245551601423
Especificidad: 0.7352941176470589
F1 score: 0.4845070422535211
Train score:  0.7400568181818182
------------------------------ TEST ------------------------------
Precision : 0.3554817275747508
Recall : 0.7278911564625851
Especificidad : 0.7356948228882834
F1 score : 0.4776785714285714
Test score:  0.7343927355278093


### Regresión logística con RFE logisticRegresion

In [127]:
regresionLogistica(df_log,y)

------------------------------ TRAIN ------------------------------
Precision: 0.3272
Recall: 0.7277580071174378
Especificidad: 0.7156862745098039
F1 score: 0.45143487858719644
Train score:  0.7176136363636364
------------------------------ TEST ------------------------------
Precision : 0.31076923076923074
Recall : 0.6870748299319728
Especificidad : 0.6948228882833788
F1 score : 0.4279661016949152
Test score:  0.6935300794551645


### Regresión logística con RFE RandomForest

In [128]:
regresionLogistica(df_ranfor,y)

------------------------------ TRAIN ------------------------------
Precision: 0.35387177352206495
Recall: 0.7562277580071174
Especificidad: 0.7376605814739688
F1 score: 0.48213272830402715
Train score:  0.740625
------------------------------ TEST ------------------------------
Precision : 0.347682119205298
Recall : 0.7142857142857143
Especificidad : 0.7316076294277929
F1 score : 0.4677060133630289
Test score:  0.7287173666288309


### Regresión logística con SelectFromModel Ridge

In [129]:
regresionLogistica(df_ridge,y)

------------------------------ TRAIN ------------------------------
Precision: 0.3259668508287293
Recall: 0.7348754448398577
Especificidad: 0.711291413116971
F1 score: 0.45161290322580644
Train score:  0.7150568181818182
------------------------------ TEST ------------------------------
Precision : 0.31722054380664655
Recall : 0.7142857142857143
Especificidad : 0.6920980926430518
F1 score : 0.43933054393305443
Test score:  0.695800227014756


### Regresión logística con SelectFromModel Gradient boosting

In [130]:
regresionLogistica(df_gb,y)

------------------------------ TRAIN ------------------------------
Precision: 0.3414239482200647
Recall: 0.7508896797153025
Especificidad: 0.7248140635564571
F1 score: 0.4694104560622914
Train score:  0.7289772727272728
------------------------------ TEST ------------------------------
Precision : 0.340625
Recall : 0.7414965986394558
Especificidad : 0.7125340599455041
F1 score : 0.4668094218415417
Test score:  0.717366628830874


# Random Forest Classifier

In [131]:
def bosqueAleatorio (df,y):
    X_train, X_test,y_train,y_test = train_test_split(df,y,test_size=0.2, random_state=24)

    num=df.select_dtypes(include=[float,int]).columns


    ranfor=RandomForestClassifier(n_estimators=300,max_depth=30,n_jobs=-1,max_leaf_nodes=20,min_samples_leaf=10,class_weight="balanced").fit(X_train,y_train)
    y_pred_train=ranfor.predict(X_train)
    y_pred_test=ranfor.predict(X_test)
    mc_train=confusion_matrix(y_train,y_pred_train)
    mc_test=confusion_matrix(y_test,y_pred_test)
    tn, fp, fn, tp = mc_train.ravel()

    precision = tp / (tp + fp)
    recall = tp / (tp + fn)
    especificidad = tn / (fp + tn)
    f1_score = 2*(precision*recall)/(precision+recall)
    print('-'*30,'TRAIN','-'*30)
    print(f'Precision: {precision}')
    print(f'Recall: {recall}')
    print(f'Especificidad: {especificidad}')
    print(f'F1 score: {f1_score}')
    print('Train score: ',ranfor.score(X_train,y_train))

    tn, fp, fn, tp = mc_test.ravel()

    precision = tp / (tp + fp)
    recall = tp / (tp + fn)
    especificidad = tn / (fp + tn)
    f1_score = 2*(precision*recall)/(precision+recall)
    print('-'*30,'TEST','-'*30)
    print(f'Precision : {precision}')
    print(f'Recall : {recall}')
    print(f'Especificidad : {especificidad}')
    print(f'F1 score : {f1_score}')
    print('Test score: ',ranfor.score(X_test,y_test))

### Random Forest con todas las características

In [132]:
bosqueAleatorio(X,y)

------------------------------ TRAIN ------------------------------
Precision: 0.580964153275649
Recall: 0.8362989323843416
Especificidad: 0.8853955375253549
F1 score: 0.6856309263311452
Train score:  0.8775568181818182
------------------------------ TEST ------------------------------
Precision : 0.5221674876847291
Recall : 0.7210884353741497
Especificidad : 0.8678474114441417
F1 score : 0.6057142857142858
Test score:  0.8433598183881952


### Random Forest con selección manual

In [133]:
bosqueAleatorio(df_manual,y)

------------------------------ TRAIN ------------------------------
Precision: 0.5303550973654066
Recall: 0.8238434163701067
Especificidad: 0.861392832995267
F1 score: 0.6452961672473867
Train score:  0.8553977272727272
------------------------------ TEST ------------------------------
Precision : 0.4794520547945205
Recall : 0.7142857142857143
Especificidad : 0.8446866485013624
F1 score : 0.5737704918032787
Test score:  0.8229284903518729


### Random Forest con Variance threshold 0.20

In [134]:
bosqueAleatorio(df_vth,y)

------------------------------ TRAIN ------------------------------
Precision: 0.5287769784172662
Recall: 0.7846975088967971
Especificidad: 0.8671399594320487
F1 score: 0.6318051575931232
Train score:  0.8539772727272728
------------------------------ TEST ------------------------------
Precision : 0.44878048780487806
Recall : 0.6258503401360545
Especificidad : 0.8460490463215259
F1 score : 0.5227272727272728
Test score:  0.8093076049943246


### Random Forest con Variance threshold 0.25

In [135]:
bosqueAleatorio(df_vth2,y)

------------------------------ TRAIN ------------------------------
Precision: 0.4638949671772429
Recall: 0.7544483985765125
Especificidad: 0.8343475321162948
F1 score: 0.5745257452574526
Train score:  0.821590909090909
------------------------------ TEST ------------------------------
Precision : 0.42342342342342343
Recall : 0.6394557823129252
Especificidad : 0.8256130790190735
F1 score : 0.5094850948509486
Test score:  0.7945516458569807


### Random Forest con SelectKbest Anova

In [136]:
bosqueAleatorio(df_anova,y)

------------------------------ TRAIN ------------------------------
Precision: 0.502262443438914
Recall: 0.7900355871886121
Especificidad: 0.8512508451656524
F1 score: 0.6141078838174274
Train score:  0.8414772727272727
------------------------------ TEST ------------------------------
Precision : 0.46255506607929514
Recall : 0.7142857142857143
Especificidad : 0.8337874659400545
F1 score : 0.5614973262032086
Test score:  0.8138479001135074


### Random Forest con SelectKbest chi2

In [137]:
bosqueAleatorio(df_chi2,y)

------------------------------ TRAIN ------------------------------
Precision: 0.5221445221445221
Recall: 0.797153024911032
Especificidad: 0.861392832995267
F1 score: 0.6309859154929578
Train score:  0.8511363636363637
------------------------------ TEST ------------------------------
Precision : 0.46261682242990654
Recall : 0.673469387755102
Especificidad : 0.8433242506811989
F1 score : 0.5484764542936287
Test score:  0.8149829738933031


### Random Forest con RFE logisticRegresion

In [138]:
bosqueAleatorio(df_log,y)

------------------------------ TRAIN ------------------------------
Precision: 0.42599620493358636
Recall: 0.798932384341637
Especificidad: 0.7954699121027722
F1 score: 0.5556930693069306
Train score:  0.7960227272727273
------------------------------ TEST ------------------------------
Precision : 0.4072727272727273
Recall : 0.7619047619047619
Especificidad : 0.7779291553133515
F1 score : 0.5308056872037915
Test score:  0.775255391600454


### Random Forest con RFE RandomForest

In [139]:
bosqueAleatorio(df_ranfor,y)

------------------------------ TRAIN ------------------------------
Precision: 0.5477629987908101
Recall: 0.806049822064057
Especificidad: 0.8735632183908046
F1 score: 0.652267818574514
Train score:  0.8627840909090909
------------------------------ TEST ------------------------------
Precision : 0.47619047619047616
Recall : 0.6802721088435374
Especificidad : 0.8501362397820164
F1 score : 0.5602240896358543
Test score:  0.8217934165720772


### Random Forest con SelectFromModel Ridge

In [140]:
bosqueAleatorio(df_ridge,y)

------------------------------ TRAIN ------------------------------
Precision: 0.41079597438243365
Recall: 0.798932384341637
Especificidad: 0.7822853279242732
F1 score: 0.5425981873111783
Train score:  0.7849431818181818
------------------------------ TEST ------------------------------
Precision : 0.38869257950530034
Recall : 0.7482993197278912
Especificidad : 0.7643051771117166
F1 score : 0.5116279069767442
Test score:  0.7616345062429057


### Random Forest con SelectFromModel Gradient boosting

In [141]:
bosqueAleatorio(df_gb,y)

------------------------------ TRAIN ------------------------------
Precision: 0.5137614678899083
Recall: 0.797153024911032
Especificidad: 0.8566599053414469
F1 score: 0.6248256624825662
Train score:  0.8471590909090909
------------------------------ TEST ------------------------------
Precision : 0.4660633484162896
Recall : 0.7006802721088435
Especificidad : 0.8392370572207084
F1 score : 0.5597826086956521
Test score:  0.8161180476730987
